In [53]:
import os
from dotenv  import load_dotenv

In [54]:
#!pip install -r requirements.txt 

In [58]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

In [6]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
file_path = "data/SDGs_Booklet_Web_En.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [9]:
data

In [ ]:
# Extracting documents: store all the text in one place
question_gen = ""
for page in data:
    question_gen += page.page_content

In [14]:
question_gen

In [33]:
# Chunking
# Recommended: Can input strings but better to convert ur string to document
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document

In [34]:
splitter_ans_gen = TokenTextSplitter(
    model_name='gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [37]:
chunk_ques_gen = splitter_ans_gen.split_text(question_gen)

In [40]:
document_ques_gen = [Document(page_content=chunk) for chunk in chunk_ques_gen]

In [42]:
splitter_ans_gen = TokenTextSplitter(
    model_name='gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [44]:
document_ans_gen = splitter_ans_gen.split_documents(document_ques_gen)

In [47]:
document_ques_gen

In [48]:
# llm
from langchain.chat_models import ChatOpenAI

In [59]:
llm_ques_gen_pipeline = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

In [60]:
prompt_template = "You are an expert instructor in {topic}, preparing a student for an upcoming exam based" \
" on {source_material}. Generate 5 critical exam-style questions covering the most essential concepts and topics " \
"from the provided material. Ensure the questions effectively reflect what the student must know to excel" 

In [62]:
from langchain.prompts import PromptTemplate

In [63]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_fields=["topic", "source_material"])

In [1]:
from langchain.chains import LLMChain

# Create the chain
question_gen_chain = LLMChain(
    llm=llm_ques_gen_pipeline,
    prompt=PROMPT_QUESTIONS
)

# Example usage
topic = "Sustainable Development Goals"
source_material = question_gen
generated_questions = question_gen_chain.run(topic=topic, source_material=source_material)
print(generated_questions)

NameError: name 'llm_ques_gen_pipeline' is not defined

In [74]:
ques_list = generated_questions.split("\n")

In [75]:
ques_list = [q for q in ques_list if q.strip() != '']

In [77]:
ques_list

In [78]:
# embeddings 
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [80]:
# vectordb
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(document_ans_gen,embeddings)

In [82]:
llm_ans_gen_pipeline = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

In [87]:
from langchain.chains import RetrievalQA
gen_chain = RetrievalQA.from_chain_type(llm = llm_ans_gen_pipeline, chain_type = "stuff",
                                         retriever = vector_store.as_retriever())

In [89]:
# ask the questions
for q in ques_list:
    answer = gen_chain.run(q)
    print(f"Question: {q}")
    print(f"Answer: {answer}")
    print("\n")
    with open("answers.txt", "a") as f:
        f.write(f"Question: {q}\n")
        f.write(f"Answer: {answer}\n")
        f.write("---------------------------------\n")